In [2]:
import xlsxwriter as xls 
import numpy as np
import pandas as pd
import pyodbc
import os
import re

In [3]:
keys=pd.read_excel('SDGs關鍵字總表.xlsx', engine='openpyxl')
writer = pd.ExcelWriter('SDGs關鍵字整理.xlsx')

for col in keys.columns:
    temp=keys[col]
    temp=temp.drop_duplicates()
    temp.to_excel(writer,sheet_name = col,index=False)
writer.save()

In [14]:
col=['PK','SDG1：終結貧窮.1',          'SDG2：終結飢餓.1',
             'SDG3：良好健康與福祉.1',          'SDG4：優質教育.1',          'SDG5：性別平等.1',
            'SDG6：潔淨水資源與衛生.1',    'SDG7：人人可負擔的永續能源.1',     'SDG8：良好工作及經濟成長.1',
         'SDG9：工業化、創新及基礎建設.1',        'SDG10：消弭不平等.1',      'SDG11：永續城鄉與社區.1',
         'SDG12：負責任的生產消費循環.1',       'SDG13：氣候變遷對策.1',         'SDG14：海洋生態.1',
               'SDG15：陸域生態.1',     'SDG16：公平、正義與和平.1',       'SDG17：全球夥伴關係.1']
t1=pd.read_excel('SDGs課程分類_婷茹.xlsx', engine='openpyxl')
t2=pd.read_excel('SDGs課程分類_昱霖.xlsx', engine='openpyxl')
t3=pd.read_excel('SDGs課程分類_聖晴.xlsx',sheet_name='總表', engine='openpyxl')

t4=pd.merge(t3,t2[col],how='left',on='PK')
t5=pd.merge(t4,t1[col],how='left',on='PK')
print(len(t5))


In [15]:
t5.to_excel('SDGs課程分類_統整.xlsx')

NameError: name 't5' is not defined

In [6]:
def is_contains_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False

In [7]:
keyword=pd.read_excel('SDGs關鍵字總表.xlsx',sheet_name='關鍵字(移除重複)')
course=pd.read_excel('1092_to_1101_class_data.xlsx')
keyflag=True
print(course)
print(keyword.columns)
for SDG in keyword.columns:
    print("<<<<<<<<<<<<<   ",SDG,"  >>>>>>>>>>>>>>>>")
    temp=keyword[SDG].dropna()
    for key in temp:
        keyflag=True;
        key=str.lower(key)
        for course_num,name,dep,schedule,syllabus,style,schema in zip(course['PK'],course['科目名稱'].fillna('無'),course['主開學系'].fillna('無'),course['每周進度'].fillna('無'),course['概述與目標'].fillna('無'),course['教學方法'].fillna('無'),course['課程規劃'].fillna('無')):
            name=str.lower(name)
            schedule=str.lower(schedule)
            syllabus=str.lower(syllabus)
            style=str.lower(style)
            schema=str.lower(schema)
            if '&' in key:
                flag=True
                sub_keys=re.split("&",key)
                for sub_key in sub_keys:
                    sub_key=sub_key.strip()
                    if (is_contains_chinese(key)==False):
                        sub_key=sub_key+' '
                    
                        if keyflag:
                            print(sub_key)
                    flag=flag and ((sub_key in name) or (sub_key in schedule) or (sub_key in syllabus) or (sub_key in style) or (sub_key in schema))
                
                keyflag=False
                if flag:
                    course.loc[course['PK']==course_num,SDG+'_程式'] = key
                    
            else:
                key=key.strip()
                if (is_contains_chinese(key)==False):
                    key=key+' '
                    
                if keyflag:
                    print(key)
                    keyflag=False
                if (key in name) or (key in schedule) or (key in syllabus) or (key in style) or (key in schema):
                    course.loc[course['PK']==course_num,SDG+'_程式'] = key
            #ok
            if SDG=='SDG4：優質教育' and dep=='師資培育中心':
                    course.loc[course['PK']==course_num,SDG+'_程式'] = '師培中心開設課程'
            if SDG=='SDG4：優質教育' and dep=='語言教學與研究中心':
                    course.loc[course['PK']==course_num,SDG+'_程式'] = '語言教學與研究中心課程'
            if SDG=='SDG4：優質教育' and dep=='寫作中心':
                    course.loc[course['PK']==course_num,SDG+'_程式'] = '寫作中心'
print("%%%%")
#d=pd.read_excel('SDG/'+school+'課程分類結果.xlsx')

#temp=pd.merge(course,d,how='left',on='PK')

#temp=temp.rename(columns={'學年度_x':'學年度','學期別_x':'學期別','課號_x':'課號','科目代碼_x':'科目代碼','科目名稱_x':'科目名稱','課程目標_x':'課程目標','課程進度_x':'課程進度','課程大綱_x':'課程大綱','開課學院_x':'開課學院','主開學系_x':'主開學系'})
#temp[['學年度','學期別','課號','科目代碼','科目名稱','課程目標','課程進度','課程大綱','開課學院','主開學系','SDG1：終結貧窮_x','SDG1：終結貧窮_y','SDG2：終結飢餓_x','SDG2：終結飢餓_y','SDG3：良好健康與福祉_x','SDG3：良好健康與福祉_y','SDG4：優質教育_x','SDG4：優質教育_y','SDG5：性別平等_x','SDG5：性別平等_y','SDG6：潔淨水資源與衛生_x','SDG6：潔淨水資源與衛生_y','SDG7：人人可負擔的永續能源_x','SDG7：人人可負擔的永續能源_y','SDG8：良好工作及經濟成長_x','SDG8：良好工作及經濟成長_y','SDG9：工業化、創新及基礎建設_x','SDG9：工業化、創新及基礎建設_y','SDG10：消弭不平等_x','SDG10：消弭不平等_y','SDG11：永續城鄉與社區_x','SDG11：永續城鄉與社區_y','SDG12：負責任的生產消費循環_x','SDG12：負責任的生產消費循環_y','SDG13：氣候變遷對策_x','SDG13：氣候變遷對策_y','SDG14：海洋生態_x','SDG14：海洋生態_y','SDG15：陸域生態_x','SDG15：陸域生態_y','SDG16：公平、正義與和平_x','SDG16：公平、正義與和平_y','SDG17：全球夥伴關係_x','SDG17：全球夥伴關係_y']].to_excel('SDG/'+school+'程式分類_名稱大綱進度.xlsx',index=False)
#course.to_excel(SDGs課程分類_統整(含程式.xlsx',index=False)
course.to_excel('SDGs課程分類_統整(測試).xlsx',index=False)
print('done')

               PK    學年度  學期別 學院代碼    開課學院 學系代碼       主開學系  主開(1為主開)  學系類型  \
0     1092DOH1002  109.0  2.0    A  人文社會學院  4A1  人文社會學院院本部       1.0  一般大學   
1     1092DOH1020  109.0  2.0    A  人文社會學院  4A1  人文社會學院院本部       1.0  一般大學   
2     1092DOH1023  109.0  2.0    A  人文社會學院  4A1  人文社會學院院本部       1.0  一般大學   
3     1092DOH1045  109.0  2.0    A  人文社會學院  4A1  人文社會學院院本部       1.0  一般大學   
4     1092DOH1046  109.0  2.0    A  人文社會學院  4A1  人文社會學院院本部       1.0  一般大學   
...           ...    ...  ...  ...     ...  ...        ...       ...   ...   
2020    1101B2446  110.0  1.0    3    護理學院  201       護理學系       1.0  一般大學   
2021    1101B2447  110.0  1.0    3    護理學院  201       護理學系       1.0  一般大學   
2022    1101B2494  110.0  1.0    3    護理學院  201       護理學系       1.0  一般大學   
2023    1101B2495  110.0  1.0    3    護理學院  201       護理學系       1.0  一般大學   
2024    1101B2577  110.0  1.0    3    護理學院  201       護理學系       1.0  一般博士   

         科目代碼          科目名稱 去年開課  \
0     DOH1002       設計基礎(二)

reading abilities 
simulink 
statistical computing language r 
writing skill 
3d繪圖軟體
ability of writing 
adobe audition cc 
adobe premiere pro cc 
ai applications 
ai in radiotherapy 
analysis 
analysis 
applications 
problem solving 
analysis 
presentation 
analysis 
projects 
analysis 
techniques 
analyze 
real data 
ansys入門
app inventor 
application 
statistical 
applications 
techniques 
applications 
theory 
experimental 
apply 
real-life situation 
approach 
manage 
business 
del ar 
vr 
del artificial intelligenc 
fintech 
del artificial intelligence 
medical 
auto cad 
autocad基本操作
basic concepts 
discussion 
basic 
linear analysis 
big data 
ai 
business strategies 
management 
c programming 
c++ 
basics 
c++ 
practices 
c++ 
strings 
function 
clinical practice 
code 
skill 
communication 
english 
culture 
communication 
multimedia 
communication 
tools 
communicative experience 
communicator 
persuation 
company 
strategies 
computer vision 
analysis 
contemporary 
discussio

cryptocurrency 
digital market 
direct marketing 
economic growth 
economic growth 
efficiency of markets 
energy 
sustainable policy 
engineer professional 
entrepreneur 
entrepreneurial 
entrepreneurship 
exporting 
financial 
financial analysis 
financial markets 
firm 
strategy 
firm strategy 
firm 
global 
firm 
innovative capabilities 
firm 
management 
firm 
strategy 
firms 
interconnect 
foreign 
bussiness 
foreign 
finance 
foreign 
market 
foreign 
trade 
futures markets 
global economy 
global marketing strategies 
global marketplace 
global 
bussiness 
global 
finance 
global 
market 
global 
trade 
green energy 
conversion 
green energy 
generation 
high tech 
plant 
human resource management 
innovation strategy 
innovation 
policy 
international business 
international 
bussiness 
international 
finance 
international 
market 
international 
trade 
internship 
machine learning 
finance 
management 
high tech firms 
management 
high tech firms 
market equilibrium 
market 

犯罪防制
犯罪偵查
白領犯罪
交易法
企業併購法
企業法規
dek 全民健保
del 全民健康保險
全球治理
刑事民事責任
刑事訴訟法
刑法
刑罰
合憲
del 在地關懷
死刑存廢
行政法
del 批判思辨
身心障礙權益保障法
併購法
兩公約
del 咖啡正義
性別平等政策
性別平等教育法
歧視＆禁止
法令分析
法令規範
法律
法律扶助
法律爭議
法律思考
法律實務
法規制度
社會公平正義
社會正義
社會參與
社會資源分配不均
金融監理
金融機構法規
契約
契約法
政策法律分析
美國反洗錢法制
美國憲法
食品法
恐怖主義
消弭性別歧視
消費者保護
消費者保護法
財經刑法
財經法
健康法政
商業賄絡
商標及著作權法
國防安全政策
國際人權
國際法
專利法
產品責任法
組織犯罪
勞工法規
勞動法
勞動法規
勞基法
勞資
勞資爭議、勞動法規
智財保護
智財訴訟
智慧財產權
智慧財產權法
著作權
著作權法
訴訟制度
集體勞動法、工會、勞動人權
del 經濟個體合作
資訊隱私權
資訊隱私權法
資通訊法律
跨國性組織犯罪
跨國法律
福利政策
精神衛生法
網路犯罪
憲法
憲法與人權
del 環境正義
聯合國身心障礙者權利公約
職業安全衛生法
轉型正義
醫事法律
醫療法
醫療法律
醫療糾紛審判
<<<<<<<<<<<<<    SDG17：全球夥伴關係   >>>>>>>>>>>>>>>>
corporate governance 
corporate 
finance 
crpd 
financial economics 
formulation of national trade policy 
fta 
international trade policy 
gats 
gatt 
global economy 
global marketplace 
global 
market 
global 
trade 
globalization and principles of management 
globalization 
international trade 
globalized 
gloobalization 
government policies 
international trade 
imf 
imf 
wb 
international commer

In [27]:

course.to_excel('SDGS課程分類/SDGs課程分類_統整(含程式.xlsx',index=False)
print('done')

done


In [4]:
course.to_excel('SDG/1101與1092課程程式分類_名稱大綱進度方法規劃.xlsx',index=False)

In [9]:
school="交大"
keyword=pd.read_excel('SDG/'+school+'關鍵字.xlsx')
course=pd.read_excel('SDG/'+school+'課程_原始資料.xlsx')
keyflag=True
print(course)
for SDG in keyword.columns:
    print("<<<<<<<<<<<<<   ",SDG,"  >>>>>>>>>>>>>>>>")
    temp=keyword[SDG].dropna()
    for key in temp:
        keyflag=True;
        key=str.lower(key)
        for course_num,name,dep,schedule,syllabus in zip(course['PK'],course['課程目標'].fillna('無'),course['主開學系'].fillna('無'),course['課程進度'].fillna('無'),course['課程大綱'].fillna('無')):
            name=str.lower(name)
            schedule=str.lower(schedule)
            syllabus=str.lower(syllabus)
            if '&' in key:
                flag=True
                sub_keys=re.split("&",key)
                for sub_key in sub_keys:
                    sub_key=sub_key.strip()
                    if (is_contains_chinese(key)==False):
                        sub_key=' '+sub_key+' '
                    
                        if keyflag:
                            print(sub_key)
                    flag=flag and ((sub_key in name) or (sub_key in schedule) or (sub_key in syllabus))
                
                keyflag=False
                if flag:
                    course.loc[course['PK']==course_num,SDG+'.1'] = key
                    
            else:
                key=key.strip()
                if (is_contains_chinese(key)==False):
                    key=' '+key+' '
                    
                if keyflag:
                    print(key)
                    keyflag=False
                if (key in name) or (key in schedule) or (key in syllabus):
                    course.loc[course['PK']==course_num,SDG+'.1'] = key
            #ok
            if SDG=='SDG4：優質教育' and dep=='師資培育中心':
                    course.loc[course['PK']==course_num,SDG+'.1'] = '師培中心開設課程'
print("%%%%")
d=pd.read_excel('SDG/'+school+'課程分類結果.xlsx')

temp=pd.merge(course,d,how='left',on='PK')

temp=temp.rename(columns={'學年度_x':'學年度','學期別_x':'學期別','課號_x':'課號','科目代碼_x':'科目代碼','科目名稱_x':'科目名稱','課程目標_x':'課程目標','課程進度_x':'課程進度','課程大綱_x':'課程大綱','開課學院_x':'開課學院','主開學系_x':'主開學系'})
temp[['學年度','學期別','課號','科目代碼','科目名稱','課程目標','課程進度','課程大綱','開課學院','主開學系','SDG1：終結貧窮_x','SDG1：終結貧窮_y','SDG2：終結飢餓_x','SDG2：終結飢餓_y','SDG3：良好健康與福祉_x','SDG3：良好健康與福祉_y','SDG4：優質教育_x','SDG4：優質教育_y','SDG5：性別平等_x','SDG5：性別平等_y','SDG6：潔淨水資源與衛生_x','SDG6：潔淨水資源與衛生_y','SDG7：人人可負擔的永續能源_x','SDG7：人人可負擔的永續能源_y','SDG8：良好工作及經濟成長_x','SDG8：良好工作及經濟成長_y','SDG9：工業化、創新及基礎建設_x','SDG9：工業化、創新及基礎建設_y','SDG10：消弭不平等_x','SDG10：消弭不平等_y','SDG11：永續城鄉與社區_x','SDG11：永續城鄉與社區_y','SDG12：負責任的生產消費循環_x','SDG12：負責任的生產消費循環_y','SDG13：氣候變遷對策_x','SDG13：氣候變遷對策_y','SDG14：海洋生態_x','SDG14：海洋生態_y','SDG15：陸域生態_x','SDG15：陸域生態_y','SDG16：公平、正義與和平_x','SDG16：公平、正義與和平_y','SDG17：全球夥伴關係_x','SDG17：全球夥伴關係_y']].to_excel('SDG/'+school+'程式分類_目標大綱進度.xlsx',index=False)
print('done')

FileNotFoundError: [Errno 2] No such file or directory: 'SDG/交大關鍵字.xlsx'